
# Patrón Singleton en Python 🐍

Este notebook explica el patrón de diseño **Singleton**, su motivación, ejemplos en Python y varios ejercicios prácticos de dificultad creciente.



## ¿Qué es el Singleton?

**Definición rápida:** Garantiza que una clase **solo tenga una instancia** y que se proporcione un **punto de acceso global** a ella.

### Cuándo usarlo:
- Cuando hay **recursos compartidos** (conexión a BD, configuración global, logger).
- Cuando quieres evitar que el programa cree varias instancias de algo que debería ser único.



## Ejemplo clásico de Singleton en Python


In [1]:

class Singleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            print("Creando nueva instancia...")
            cls._instance = super().__new__(cls)
        return cls._instance

# Probar el Singleton
s1 = Singleton()
s2 = Singleton()

print(s1 is s2)  # True


Creando nueva instancia...
True



## Ejercicio 1: Implementa tu propio Singleton

Crea una clase llamada `LoggerSingleton` que guarde un registro de mensajes en una lista interna `logs`. La clase debe ser un Singleton.

Prueba que todas las instancias comparten la misma lista de logs.


In [2]:

# ✅ Solución:

class LoggerSingleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.logs = []
        return cls._instance

# Test
logger1 = LoggerSingleton()
logger2 = LoggerSingleton()

logger1.logs.append("Mensaje 1")
logger2.logs.append("Mensaje 2")

print(logger1.logs)  # Ambas instancias comparten la misma lista
print(logger1 is logger2)  # True


['Mensaje 1', 'Mensaje 2']
True



## Variante: Borg Pattern (Monostate)

Permite múltiples instancias, pero **todas comparten el mismo estado**.


In [3]:

class Borg:
    _shared_state = {}

    def __new__(cls):
        obj = super().__new__(cls)
        obj.__dict__ = cls._shared_state
        return obj

# Test
b1 = Borg()
b2 = Borg()

b1.x = 42
print(b2.x)  # 42, comparten el estado

print(b1 is b2)  # False: son instancias diferentes pero con estado compartido


42
False



## Ejercicio 2: Singleton con método de clase

Crea un Singleton llamado `Config` pero esta vez usando un **classmethod** para acceder a la instancia, no con `__new__`.

Debe tener un atributo `settings` que sea un diccionario.


In [4]:

# ✅ Solución:

class Config:
    _instance = None

    def __init__(self):
        self.settings = {}

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = cls()
        return cls._instance

# Test
c1 = Config.get_instance()
c2 = Config.get_instance()

c1.settings['debug'] = True
print(c2.settings)  # {'debug': True}
print(c1 is c2)  # True


{'debug': True}
True



## Ejercicio 3: Singleton Thread-safe

Modifica el siguiente código para que el Singleton sea **seguro frente a múltiples hilos** (Thread-safe).


In [14]:
import threading

class SafeSingleton:
    """
    Implementación de Singleton *thread-safe* (segura frente a múltiples hilos).

    Idea:
    - La primera vez que cualquier hilo ejecuta SafeSingleton(), se crea una instancia.
    - Las siguientes llamadas devuelven EXACTAMENTE la misma (misma dirección de memoria).
    - Un candado (`_lock`) protege la sección crítica para evitar que dos hilos
      construyan la instancia a la vez.
    """

    # Referencia a la única instancia (se comparte entre todos los hilos).
    _instance = None

    # Candado de clase (una sóla copia) que protege la construcción simultánea.
    _lock = threading.Lock()

    def __new__(cls):
        """
        __new__ se ejecuta ANTES de __init__ y es responsable de crear el objeto.
        Implementamos el patrón “double-checked locking”:

        1) Miramos si ya existe la instancia (sin bloquear) → fast-path.
        2) Si no existe, adquirimos el lock.
        3) Comprobamos de nuevo (otro hilo podría haberla creado mientras
           esperábamos el lock).
        4) Creamos la instancia sólo si sigue siendo None.
        """
        if cls._instance is None:                 # 1ª comprobación (sin lock)
            with cls._lock:                       # sección crítica
                if cls._instance is None:         # 2ª comprobación (con lock)
                    cls._instance = super().__new__(cls)
        return cls._instance


# Test multi-hilo

def create_instance():
    """Función destinada a cada hilo: solicita la instancia y muestra su id()."""
    s = SafeSingleton()
    print(f"Instancia: {id(s)}")


# Lanzamos 5 hilos que intentan crear la instancia al mismo tiempo.
threads = [threading.Thread(target=create_instance) for _ in range(5)]

# Iniciar los hilos (start es asíncrono: cada hilo ejecuta create_instance).
[t.start() for t in threads]





Instancia: 127662201655648
Instancia: 127662201655648
Instancia: 127662201655648
Instancia: 127662201655648
Instancia: 127662201655648


[None, None, None, None, None]


## Ejercicio 4: Singleton como decorador

Crea un decorador `singleton` que convierta **cualquier clase** en Singleton.


In [6]:

# ✅ Solución:

def singleton(cls):
    instances = {}

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

@singleton
class Settings:
    pass

s1 = Settings()
s2 = Settings()
print(s1 is s2)  # True


True
